In [1]:
import numpy as np
import pandas as pd
import random

In [4]:
file_path = '../bar_movement/data/'

In [5]:
df = pd.read_csv(file_path + 'Oanda_Eur_Usd_M5_2021-2022_test.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [6]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2021-10-14 06:00:00   1.15886   1.15931  1.15883    1.15925   1.15900   
1 2021-10-14 06:05:00   1.15925   1.15948  1.15921    1.15948   1.15939   
2 2021-10-14 06:10:00   1.15947   1.15957  1.15917    1.15954   1.15961   
3 2021-10-14 06:15:00   1.15956   1.15981  1.15950    1.15968   1.15970   
4 2021-10-14 06:20:00   1.15968   1.15975  1.15945    1.15954   1.15980   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  \
0   1.15945  1.15897    1.15941   1.15893   1.15938  1.15890    1.15933   
1   1.15962  1.15934    1.15962   1.15932   1.15955  1.15928    1.15955   
2   1.15971  1.15931    1.15969   1.15954   1.15964  1.15924    1.15962   
3   1.15994  1.15963    1.15983   1.15963   1.15988  1.15956    1.15976   
4   1.15987  1.15959    1.15968   1.15974   1.15981  1.15952    1.15961   

   Volume  
0     281  
1     278  
2     348  
3     280  
4     290  
--------------------------

In [7]:
def psar(barsdata, iaf=0.02, maxaf=0.2):
    length = len(barsdata)
    high = list(barsdata['Mid_High'])
    low = list(barsdata['Mid_Low'])
    close = list(barsdata['Mid_Close'])
    psar = close[0:len(close)]
    bull = True
    af = iaf
    hp = high[0]
    lp = low[0]
    for i in range(2, length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
    return psar


def atr(barsdata, lookback=14):
    high_low = barsdata['Mid_High'] - barsdata['Mid_Low']
    high_close = np.abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    low_close = np.abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).sum() / lookback


def rsi(barsdata, periods=14):
    close_delta = barsdata['Mid_Close'].diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    return rsi

  
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth


def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def stoch_rsi(data, k_window=3, d_window=3, window=14):
    min_val = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()

    stoch = ((data - min_val) / (max_val - min_val)) * 100

    slow_k = stoch.rolling(window=k_window, center=False).mean()

    slow_d = slow_k.rolling(window=d_window, center=False).mean()

    return slow_k, slow_d

def n_macd(macd, macdsignal, lookback=50):
    n_macd = 2 * (((macd - macd.rolling(lookback).min()) / (macd.rolling(lookback).max() - macd.rolling(lookback).min()))) - 1
    n_macdsignal = 2 * (((macdsignal - macdsignal.rolling(lookback).min()) / (macdsignal.rolling(lookback).max() - macdsignal.rolling(lookback).min()))) - 1

    return n_macd, n_macdsignal

def chop(df, lookback=14):
    atr1 = atr(df, lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def bar_lengths(bar_lens, window=36):
    return bar_lens.rolling(window=window).mean(), bar_lens.rolling(window=window).std()

def sar_lengths(opens, sars, window=36):
    diffs = abs(opens - sars.shift(1))

    return diffs.rolling(window=window).mean(), diffs.rolling(window=window).std()

def range_high_lows(df, i, n_histograms=6):
    green, red = False, False
    curr_color, curr_streak = None, 0
    high, low = -np.inf, np.inf

    while i >= 0:
        hist = df.loc[df.index[i], 'macdhist']
        curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
        new_color = 'green' if hist > 0 else 'red'

        if curr_color != new_color:
            curr_color = new_color
            curr_streak = 0

            if green and red:
                break

        curr_streak += 1

        if curr_streak >= n_histograms:
            high = max(high, curr_high)
            low = min(low, curr_low)

            if curr_color == 'red':
                red = True

            else:
                green = True

        i -= 1

    return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan

# def range_high_lows(df, i):
#     green, red = False, False
#     curr_color, curr_streak = None, 0
#     high, low = -np.inf, np.inf

#     while i >= 0:
#         hist = df.loc[df.index[i], 'macdhist']
#         curr_high, curr_low = df.loc[df.index[i], ['Mid_High', 'Mid_Low']]
#         new_color = 'green' if hist > 0 else 'red'

#         if curr_color != new_color:
#             curr_color = new_color
#             curr_streak = 0

#             if green and red:
#                 break

#         curr_streak += 1

#         if curr_streak >= 3:
#             if curr_color == 'red':
#                 low = min(low, curr_low)
#                 red = True

#             else:
#                 high = max(high, curr_high)
#                 green = True

#         i -= 1

#     return high, low if (high > -np.inf and low < np.inf) else np.nan, np.nan

def fib_bb(highs, lows, closes, volumes, length=200, mult=3):
    hlc3 = (highs + lows + closes) / 3
    basis = (hlc3 * volumes).rolling(window=length).sum() / volumes.rolling(window=length).sum()
    dev = hlc3.rolling(window=length).std() * mult
    upper236 = basis + (0.236 * dev)
    upper382 = basis + (0.382 * dev)
    upper5 = basis + (0.5 * dev)
    upper618 = basis + (0.618 * dev)
    upper764 = basis + (0.764 * dev)
    upper1 = basis + (1 * dev)
    lower236 = basis - (0.236 * dev)
    lower382 = basis - (0.382 * dev)
    lower5 = basis - (0.5 * dev)
    lower618 = basis - (0.618 * dev)
    lower764 = basis - (0.764 * dev)
    lower1 = basis - (1 * dev)

    return upper236, upper382, upper5, upper618, upper764, upper1, lower236, lower382, lower5, lower618, lower764, lower1

def bb(barsdata, length=30, mult=2.0):
    m_avg = barsdata['Mid_Close'].rolling(window=length).mean()
    m_std = barsdata['Mid_Close'].rolling(window=length).std(ddof=0)
    upper_bb = m_avg + mult * m_std
    lower_bb = m_avg - mult * m_std

    return lower_bb, m_avg, upper_bb

def rsi_divergence(barsdata, rsi_len=9, lbR=3, lbL=1):
    rsi_vals = rsi(barsdata, periods=rsi_len)


In [8]:
# Add technical indicators (for additional features)
df['lower_bb'], df['mid_bb'], df['upper_bb'] = bb(df) 

df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df)

                     Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0     2021-10-14 08:25:00   1.16131   1.16150  1.16128    1.16145   1.16145   
1     2021-10-14 08:30:00   1.16143   1.16169  1.16133    1.16167   1.16157   
2     2021-10-14 08:35:00   1.16169   1.16175  1.16151    1.16154   1.16181   
3     2021-10-14 08:40:00   1.16156   1.16172  1.16141    1.16154   1.16169   
4     2021-10-14 08:45:00   1.16153   1.16182  1.16151    1.16178   1.16165   
...                   ...       ...       ...      ...        ...       ...   
74846 2022-10-14 05:35:00   0.97806   0.97832  0.97789    0.97824   0.97822   
74847 2022-10-14 05:40:00   0.97827   0.97832  0.97789    0.97791   0.97845   
74848 2022-10-14 05:45:00   0.97790   0.97819  0.97776    0.97811   0.97808   
74849 2022-10-14 05:50:00   0.97812   0.97818  0.97780    0.97808   0.97827   
74850 2022-10-14 05:55:00   0.97805   0.97805  0.97744    0.97752   0.97821   

       Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_

In [10]:
df.loc[df.index[-10:], ['Date', 'lower_bb', 'mid_bb', 'upper_bb']]

,Date,lower_bb,mid_bb,upper_bb
74841,2022-10-14 05:10:00,0.977958,0.978754,0.979550
74842,2022-10-14 05:15:00,0.977947,0.978731,0.979516
74843,2022-10-14 05:20:00,0.977881,0.978696,0.979510
74844,2022-10-14 05:25:00,0.977810,0.978659,0.979508
74845,2022-10-14 05:30:00,0.977768,0.978630,0.979492
74846,2022-10-14 05:35:00,0.977748,0.978612,0.979476
74847,2022-10-14 05:40:00,0.977704,0.978576,0.979449
74848,2022-10-14 05:45:00,0.977684,0.978548,0.979413
74849,2022-10-14 05:50:00,0.977664,0.978518,0.979372
74850,2022-10-14 05:55:00,0.977570,0.978478,0.979386
